In [1]:
using PastaQ

In [2]:
function PastaQ.gate(::GateName"R"; theta::Real, phi::Real)
    [
        cos(theta/2)    (-im * exp(-im * phi) * sin(theta/2))
        (-im * exp(im * phi) * sin(theta/2))     cos(theta/2)
    ]
end

In [3]:
function PastaQ.gate(::GateName"M"; Theta::Real)
    [
        cos(Theta)    0    0    (-im * sin(Theta))
        0    cos(Theta)    (-im * sin(Theta))    0
        0    (-im * sin(Theta))    cos(Theta)    0
        (-im * sin(Theta))    0    0    cos(Theta)
    ]
end

In [4]:
function PastaQ.gate(::GateName"Z";)
    [
        1.00    0
        0     -1.00
    ]
end

In [5]:
function run(N, depth)
    # Random circuit.
    gates = Vector{Tuple}[]
    
    for i in 1:depth
        one_qubit_layer = Tuple[]
        two_qubit_layer = Tuple[]

        for j in 1:N
            gate = ("R", j, (theta=2pi*rand(), phi=2pi*rand()))
            push!(one_qubit_layer, gate)
        end

        # Alternate start qubit for pairs.
        idx_first = i % 2 + 1

        for j in idx_first:2:(N-1)
            gate = ("M", (j, j+1), (Theta=2pi*rand(),))
            push!(two_qubit_layer, gate)
        end
        
        push!(gates, one_qubit_layer)
        push!(gates, two_qubit_layer)
    end
    z_layer = Tuple[]
    for j in 1:N
        gate = ("Z", j)
        push!(z_layer, gate)
    end
    push!(gates, z_layer)
    psi = runcircuit(N, gates)
end

run (generic function with 1 method)

In [6]:
using ITensors

In [7]:
function meaZure(psi)
    for j=1:length(psi)
      orthogonalize!(psi,j)

      s = siteind(psi,j)
      val = scalar(psi[j]*op(s,"Z")*dag(prime(psi[j],s)))

      println("$j $val")
    end
end

meaZure (generic function with 1 method)

In [8]:
psi=run(2,2)
meaZure(psi)

1 -0.12373426950018346 + 0.0im
2 -0.308785892385869 + 0.0im
